This document will contain the template and codes for creating master data dictionary , carrying out basic and advanced
text analytics to prepare training set data be be feeeded into an machine learning supervised model to classify
product descriptions into multi level multi class classification/categories. 

Creating the Master Data Dictionary of Product Master Classes (Nouns)
=====================================


In [8]:
from sklearn.externals import joblib 
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import nltk

In [19]:
count_vect_noun = CountVectorizer()
count_vect_noun._validate_vocabulary()
clf_noun = joblib.load('saved_model_noun.pkl')  
clf_modifiers = joblib.load('saved_model_modifiers.pkl')  
count_vect_noun = joblib.load('count_vect_noun.pkl')
data_dictionary_mapping = joblib.load('data_dictionary_mapping.pkl') 

In [20]:
def noun_identifier(test_sample):
    inp = cleaning_input_text(test_sample)
    dict_noun_modifier={}
    noun_value = ''
    modifier_value=''
    noun = clf_noun.predict(count_vect_noun.transform([inp]))
    noun_value=''.join(noun)
    modifier = clf_modifiers.predict(count_vect_noun.transform([inp]))
    modifier_value=''.join(modifier)
    dict_noun_modifier = {'noun':noun_value,'modifier':modifier_value}
    return dict_noun_modifier


def cleaning_input_text(text):
    tokens = nltk.word_tokenize(text)
    all_words_bag_temp_1 = nltk.pos_tag(tokens)
    all_words=[]
    for j in range(len(all_words_bag_temp_1)):
        if(all_words_bag_temp_1[j][1] not in ['JJ','MD','PRP','CD','POS','IN'] and all_words_bag_temp_1[j][0].isalpha()):
            all_words.append(all_words_bag_temp_1[j][0])
    clean_text=' '.join(all_words)
    clean_text_1 = clean_text
    del clean_text
    return clean_text_1

In [25]:
def testing_module(choice,test_input):
    if (choice == '1'):
        df_raw_test = pd.read_csv("Test_Data.csv",encoding = "latin")
        df_raw_test.reset_index(drop=True, inplace=True)
        output_df = pd.DataFrame(columns=['Prod_ID','Material_Desc','Noun','Modifier'])
        counter1=df_raw_test.shape[0]
        
        for i in range(counter1):
            if(i%1000 == 0):
                print(i,'Inside 1 if loop')
            prod_id = int(1000 + i)
            noun_modifier_dict = noun_identifier(df_raw_test['Long_Desc'][i])
            noun_value=noun_modifier_dict['noun']
            modifier_value = noun_modifier_dict['modifier']
            concat = noun_value + " | " + modifier_value
            output_df = output_df.append({'Prod_ID':prod_id,'Material_Desc':df_raw_test['Long_Desc'][i],'Noun':noun_value,'Modifier':modifier_value},ignore_index=True)
            
        out_df_final = pd.DataFrame()
        output_char_value_df_temp = pd.DataFrame()
        df1 = pd.DataFrame()
        counter = df_raw_test.shape[0]
        dummy_data = {'Dummy': [np.NaN]}
        output_char_value_df_dummy = pd.DataFrame(data=dummy_data)
        #characteristic_list = set(data_dictionary_mapping.iloc[:,2])
        for i in range(counter):
            if(i%1000 == 0):
                print(i,'Inside 2 if loop')
            noun = output_df['Noun'][i]
            modifier = output_df['Modifier'][i]
            char_list = data_dictionary_mapping[(data_dictionary_mapping['Noun'] == noun) & (data_dictionary_mapping['Modifier'] == modifier) ]
            char_values_all = char_list['Characteristic'].unique()
            output_char_value_df = pd.DataFrame(columns=char_values_all)
            counter1 = len(char_values_all)
            for j in range(counter1):
                char_value = char_values_all[j]
                char_flag=None
                word_list=char_value.split()
                search = word_list[0]
                prod_desc=output_df['Material_Desc'][i]
                clean_string = prod_desc.replace(':', ' ').replace('-', ' ')
                list_of_words = clean_string.split()
                if (search in list_of_words):
                    last_index =list_of_words.index(search) + len(word_list)
                    if (len(list_of_words) > last_index):
                        char_flag = list_of_words[list_of_words.index(search) + len(word_list)]
                        output_char_value_df=output_char_value_df.append({char_value:char_flag},ignore_index=True)
                if (char_flag is None):
                    output_char_value_df=output_char_value_df.append({char_value:'NA'},ignore_index=True)
            output_char_value_df = output_char_value_df.fillna(method='ffill')
            output_char_value_df = output_char_value_df.tail(1)

            if (output_char_value_df.empty):
                col = output_char_value_df_dummy.columns
                for k in range(output_char_value_df_dummy.shape[1]):
                    col_name = col[k]
                    output_char_value_df.loc[0,col_name] = np.NaN
            output_char_value_df_temp = output_char_value_df_temp.append(output_char_value_df)
            output_char_value_df=output_char_value_df.iloc[0:0]
        
        output_char_value_df_temp.reset_index(drop=True, inplace=True)
        output_df.reset_index(drop=True, inplace=True)
        final_output_df = pd.concat([output_df,output_char_value_df_temp],axis=1)
        new = pd.DataFrame()
        new = final_output_df.iloc[:,5:]
        df_temp1 = final_output_df.loc[:,'Prod_ID':'Modifier']
        df = pd.DataFrame()
        
        for i in range(new.shape[0]):
            if(i%1000 == 0):
                print(i,'Inside 3 if loop')       
            short_desc=''
            long_desc=''
            val=''
            for j in new.columns:
                val = final_output_df.loc[i,j]
                if (val != 'NA'):
                    if(not pd.isnull(val)):
                        short_desc = str(short_desc) + str(val) + ','
                        long_desc =  str(long_desc) + str(j) + ' : ' + str(val) + '\n'
            df = df.append({'short_description':short_desc,'long_description':long_desc},ignore_index=True)
        
        
        df_formatter=pd.concat([df_temp1,df],axis=1)
        
        modifier_dict = pd.read_csv('modifier_abbreviation_dict.csv')
        modifier_dict.reset_index(drop=True, inplace=True)
        for i in range(df_formatter.shape[0]):
            if(i%1000 == 0):
                print(i,'Inside 4 if loop')

            new_long_desc = "*** " + str(df_formatter.loc[i,'Noun']) + ' , ' + str(df_formatter.loc[i,'Modifier']) + " *** " + '\n\n' + str(df_formatter.loc[i,'long_description'])
            modi_search = df_formatter.loc[i,'Modifier']
            if(modi_search in list(modifier_dict['Modifier'])):
                short_modifier =modifier_dict[modifier_dict['Modifier'] == modi_search ]['Abbreviation'].values[0]
                new_short_desc =  str(df_formatter.loc[i,'Noun']) + ' , ' + str(short_modifier) + ': ' + str(df_formatter.loc[i,'short_description'])
                new_short_desc = new_short_desc[:-1]
            else:
                new_short_desc = str(df_formatter.loc[i,'Noun']) + ',' + str(df_formatter.loc[i,'Modifier']) + ':' + str(df_formatter.loc[i,'short_description'])
                new_short_desc = new_short_desc[:-1]
            df_formatter.loc[i,'long_description'] = new_long_desc
            df_formatter.loc[i,'short_description'] = new_short_desc
        
        df_exception = pd.DataFrame()
        for i in range(df_formatter.shape[0]):
            if(i%1000 == 0):
                print(i,'Inside 5 if loop')

            mat_desc=df_temp1.loc[i,'Material_Desc'].replace(',',' ')
            #mat_desc= mat_desc.split()
            exception = [w.upper() for w in nltk.word_tokenize(mat_desc) if w not in nltk.word_tokenize(df_formatter.loc[i,'long_description'])]
            exception = ' '.join(exception)
            if (len(exception)>0):
                df_exception = df_exception.append({'exception': exception},ignore_index=True)
            else:
                df_exception = df_exception.append({'exception': 'No exception found'},ignore_index=True)
        
            
        output_dataframe=pd.DataFrame()
        new_output_df=pd.DataFrame()
        new_output_df=final_output_df.iloc[:,4:]
        output_dataframe = pd.concat([df_formatter,df_exception,new_output_df],axis=1)
        output_dataframe.to_csv('final_output_df_file_import.csv')
        display(output_dataframe)
        print('Please check you local file system for output generated : File ----> final_output_df_file_import.csv')
        return output_dataframe

    if (choice == '2'):
        output_df = pd.DataFrame(columns=['Prod_ID','Material_Desc','Noun','Modifier'])
        for i in range(1):
            prod_id = int(1000 + i)
            noun_modifier_dict = noun_identifier(test_input)
            noun_value = noun_modifier_dict['noun']
            modifier_value = noun_modifier_dict['modifier']
            output_df = output_df.append({'Prod_ID':prod_id,'Material_Desc':test_input,'Noun':noun_value,'Modifier':modifier_value},ignore_index=True)

        def ngram_model(text,n_val,char_value):
            from nltk import ngrams
            sentence = text
            n = n_val
            ngrams = ngrams(sentence.split(), n)
            for grams in ngrams:
                str1 = " ".join(list(grams))
                return_val='NA'
                if (str1.upper() == char_value.upper()):
                    return_val=char_value.upper()
                    return (return_val)
                    break
                else:
                    return_val='NA'

        out_df_final = pd.DataFrame()
        output_char_value_df_temp = pd.DataFrame()
        df1 = pd.DataFrame()
        dummy_data = {'Dummy': [np.NaN]}
        output_char_value_df_dummy = pd.DataFrame(data=dummy_data)
        for i in range(1):
            characteristic_list = set(data_dictionary_mapping.iloc[:,2])
            noun = output_df['Noun'][i]
            modifier = output_df['Modifier'][i]
            char_list= data_dictionary_mapping[data_dictionary_mapping['Noun'] == noun ]
            char_list= char_list[char_list['Modifier'] == modifier ]
            char_values_all = char_list['Characteristic'].unique()
            output_char_value_df = pd.DataFrame(columns=char_values_all)
            word_tokens = [w for w in nltk.word_tokenize(output_df['Material_Desc'][i])]
            counter1 = len(char_values_all)
            for j in range(counter1):
                char_value = char_values_all[j]
                values = char_list[char_list['Characteristic'] == char_value]['TheValue']
                for k in values:
                    n_gram = len(k.split())
                    char_flag = ngram_model(output_df['Material_Desc'][i],n_gram,k)
                    if (char_flag is not None):
                        output_char_value_df=output_char_value_df.append({char_value:char_flag},ignore_index=True)
                        break
                if (char_flag is None):
                    word_list=char_value.split()
                    search = word_list[0]
                    prod_desc=output_df['Material_Desc'][i]
                    clean_string1 = prod_desc.replace(':', ' ')
                    clean_string = clean_string1.replace('-', ' ')
                    list_of_words = clean_string.split()
                    if (search in list_of_words):
                        last_index =list_of_words.index(search) + len(word_list)
                        if (len(list_of_words) > last_index):
                            char_flag = list_of_words[list_of_words.index(search) + len(word_list)]
                            output_char_value_df=output_char_value_df.append({char_value:char_flag},ignore_index=True)
                if (char_flag is not None):
                    output_char_value_df=output_char_value_df.append({char_value:char_flag},ignore_index=True)
                else:
                    output_char_value_df=output_char_value_df.append({char_value:'NA'},ignore_index=True)
            output_char_value_df = output_char_value_df.fillna(method='ffill')
            output_char_value_df = output_char_value_df.tail(1)
            if (output_char_value_df_dummy.empty):
                col = output_char_value_df_dummy.columns
                for k in range(output_char_value_df_dummy.shape[1]):
                    col_name = col[k]
                    output_char_value_df.loc[0,col_name] = 'NaN'
            output_char_value_df_temp = output_char_value_df_temp.append(output_char_value_df)
            output_char_value_df=output_char_value_df.iloc[0:0]

        output_char_value_df_temp.reset_index(drop=True, inplace=True)
        output_df.reset_index(drop=True, inplace=True)
        final_output_df = pd.concat([output_df,output_char_value_df_temp],axis=1)    
        new = pd.DataFrame()
        new = final_output_df.iloc[:,4:]
        df_temp1 = final_output_df.loc[:,'Prod_ID':'Modifier']
        df = pd.DataFrame()
        for i in range(new.shape[0]):
            short_desc=''
            long_desc=''
            gskt = final_output_df.loc[i,'Noun']
            modi = final_output_df.loc[i,'Modifier']
            for j in new.columns:
                val = final_output_df.loc[i,j]
                if (val != 'NA'):
                    if(not pd.isnull(val)):
                        short_desc = str(short_desc) + str(val) + ','
                        long_desc = str(long_desc) + str(j) + ' : ' + str(val) + '\n'
            df = df.append({'short_description':short_desc,'long_description':long_desc},ignore_index=True)
        df_formatter=pd.concat([df_temp1,df],axis=1)
        modifier_dict = pd.read_csv('modifier_abbreviation_dict.csv')
        modifier_dict.reset_index(drop=True, inplace=True)

        for i in range(df_formatter.shape[0]):
            new_long_desc = "*** " + str(df_formatter.loc[i,'Noun']) + ',' + str(df_formatter.loc[i,'Modifier']) + " ***" + '\n\n' + str(df_formatter.loc[i,'long_description'])
            modi_search = df_formatter.loc[i,'Modifier']
            if(modi_search in list(modifier_dict['Modifier'])):
                short_modifier =modifier_dict[modifier_dict['Modifier'] == modi_search ]['Abbreviation'].values[0]
                new_short_desc =  str(df_formatter.loc[i,'Noun']) + ',' + str(short_modifier) + ':' + str(df_formatter.loc[i,'short_description'])
                new_short_desc = new_short_desc[:-1]
                print(new_short_desc,'\n\n')
            else:
                new_short_desc = str(df_formatter.loc[i,'Noun']) + ',' + str(df_formatter.loc[i,'Modifier']) + ':' + str(df_formatter.loc[i,'short_description'])
                new_short_desc = new_short_desc[:-1]
                print(new_short_desc,'\n\n')
            df_formatter.loc[i,'long_description'] = new_long_desc
            df_formatter.loc[i,'short_description'] = new_short_desc

        df_exception = pd.DataFrame()
        for i in range(df_formatter.shape[0]):
            if(i%1000 == 0):
                print(i,'Inside 5 if loop')

            mat_desc=df_temp1.loc[i,'Material_Desc'].replace(',',' ')
            #mat_desc= mat_desc.split()
            exception = [w.upper() for w in nltk.word_tokenize(mat_desc) if w not in nltk.word_tokenize(df_formatter.loc[i,'long_description'])]
            exception = ' '.join(exception)
            if (len(exception)>0):
                df_exception = df_exception.append({'exception': exception},ignore_index=True)
            else:
                df_exception = df_exception.append({'exception': 'No exception found'},ignore_index=True)

        new_output_df=pd.DataFrame()
        new_output_df=final_output_df.iloc[:,4:]
        output_dataframe = pd.concat([df_formatter,df_exception,new_output_df],axis=1)
        output_dataframe.to_csv('final_output_df_text_inp.csv')
        print('Please check you local file system for output generated : File ----> final_output_df_text_inp.csv')
        return output_dataframe

In [26]:
print("Select operation to input a test sample")
print("1.File import")
print("2.Manual Text input")

Select operation to input a test sample
1.File import
2.Manual Text input


In [30]:
import warnings
warnings.filterwarnings('ignore')

import time
start = time.time()

choice = input("Enter choice(1/2):")
if choice == '1':
    print('Make sure you have feeded your input in the Test_Data.csv')
    test_input=''
    pred_output_df = pd.DataFrame()
    pred_output_df = testing_module(choice,test_input.upper())
if choice == '2':
    #test_input = input('Enter your test sample\n')
    test_input = "SPIRAL WOUND GASKET SIZE: 1""X 900/1500 LBS;WITH EXTERNAL GUIDE RING (STYLE-CG), WINDING MATERIAL:316L FILLER MATERIAL GRAPHITE AS PER ASME B 16.20 TO FIT FOR ANSI B 16.5 FLANGES"
    pred_output_df = testing_module(choice,test_input.upper())
    #display(pred_output_df)
end = time.time()
temp = end-start
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes
print(hours,':',minutes,':',seconds)

Enter choice(1/2):1
Make sure you have feeded your input in the Test_Data.csv
0 Inside 1 if loop
0 Inside 2 if loop
0 Inside 3 if loop
0 Inside 4 if loop
0 Inside 5 if loop


,Prod_ID,Material_Desc,Noun,Modifier,long_description,short_description,exception,AMPERAGE,BRAKE,ENCLOSURE,...,POWER,PRESSURE RATING,SERVICE FACTOR,SHAFT SIZE,SIZE,SPEED,STYLE,THICKNESS,VOLTAGE,WINDING MATERIAL
0,1000,"MOTOR,ELECTRIC,15HP,190/380V,1475,50HZ\nINDUCT...",MOTOR,AC,"*** MOTOR , AC *** \n\nFREQUENCY : 50\nPHASE :...","MOTOR,AC:50,3,15,OPPOSITE,1475,190/380",ELECTRIC 15HP 190/380V 50HZ INDUCTION FURNACE ...,NA,NA,NA,...,15,NaN,NA,OPPOSITE,NaN,1475,NaN,NaN,190/380,NaN
1,1001,"LSDP CONTROL\nSPIRAL WOUND GASKET 24"""""""" X 300...",GASKET,SPIRAL WOUND,"*** GASKET , SPIRAL WOUND *** \n\nFILLER MATER...","GASKET , SPL WND: CERAMIC,OUTER,23,3/4"""",3/4""""",LSDP CONTROL 24 X 300 # RF X 4.5THK . SS 316L ...,NaN,NaN,NaN,...,NaN,NA,NaN,NaN,"3/4""""",NaN,NA,NA,NaN,NA
2,1002,"MOTOR,22 KW,400-690 KW,DFV180L4/BM/HR/TH\nNO G...",MOTOR,AC,"*** MOTOR , AC *** \n\nPHASE : 3\nSPEED : 1465...","MOTOR,AC:3,1465,400",22 KW 400-690 KW DFV180L4/BM/HR/TH NO GEAR BOX...,NA,NA,NA,...,NA,NaN,NA,NA,NaN,1465,NaN,NaN,400,NaN
3,1003,GASKET SPIRAL WOUND SIZE GRAPHITE FILLER MATER...,GASKET,SPIRAL WOUND,"*** GASKET , SPIRAL WOUND *** \n\nFILLER MATER...","GASKET , SPL WND: INNER,MONEL,STEEL,10"""""""",150...",LB 10-3/8 ID X 15 OD CG FURON RW LAMONS WR C N...,NaN,NaN,NaN,...,NaN,1500,NaN,NaN,GRAPHITE,NaN,FLEXITALLIC,"1/16""""""""",NaN,HASTELLOY
4,1004,"MOTOR,7.5 KW,400/690 V D/Y,14.3/8.27A\nMOTOR F...",MOTOR,AC,"*** MOTOR , AC *** \n\nFREQUENCY : 50/60\nPOWE...","MOTOR,AC:50/60,7.5,1455/1760,400/690",KW V D/Y 14.3/8.27A FOR PASTE COOLER HYDRAULIC...,NA,NA,NA,...,7.5,NaN,NA,NA,NaN,1455/1760,NaN,NaN,400/690,NaN
5,1005,"LSDP CONTROL\nSPIRAL WOUND GASKET 8"""""""" X 150#...",GASKET,SPIRAL WOUND,"*** GASKET , SPIRAL WOUND *** \n\nFILLER MATER...","GASKET , SPL WND: GRAPHITE,OUTER,MATERIAL,150,...",LSDP CONTROL 8 X # RF X 4.5THK . 316L S.W.G FI...,NaN,NaN,NaN,...,NaN,150,NaN,NaN,NA,NaN,LAMONS,"1/8""""",NaN,SS
6,1006,"MOTOR,2.2 KW,400 V,KA57TDV100M4BMG2HF\nMANUFAC...",MOTOR,AC,"*** MOTOR , AC *** \n\nPHASE : 3\nSPEED : 1400...","MOTOR,AC:3,1400/47,400",2.2 KW V KA57TDV100M4BMG2HF MANUFACTURER INFO ...,NA,NA,NA,...,NA,NaN,NA,NA,NaN,1400/47,NaN,NaN,400,NaN
7,1007,"GASKET,SPIRAL WOUND,RWI-RJ,2""\nFILLER MATERIAL...",GASKET,SPIRAL WOUND,"*** GASKET , SPIRAL WOUND *** \n\nFILLER MATER...","GASKET , SPL WND: PTFE,SS304,STEEL,2"""""""""""""""",RWI",RWI-RJ FOR REPLACEMENT OF JOINT TYPE RWI-RJ RA...,NaN,NaN,NaN,...,NaN,NA,NaN,NaN,"2""""""""""""""""",NaN,RWI,NA,NaN,NA
8,1008,"MOTOR,ELECTRIC,0.49KW,VEP15400800\nMOTOR_ELECT...",MOTOR,AC,"*** MOTOR , AC *** \n\nSHAFT SIZE : DIRECT\nSP...","MOTOR,AC:DIRECT,1500RPM,230/400V",ELECTRIC 0.49KW VEP15400800 MOTOR_ELECTRICAL,NA,NA,NA,...,NA,NaN,NA,DIRECT,NaN,1500RPM,NaN,NaN,230/400V,NaN
9,1009,"GASKET , SPIRAL WOUND FILLER MATERIAL GRAPHI...",GASKET,SPIRAL WOUND,"*** GASKET , SPIRAL WOUND *** \n\nFILLER MATER...","GASKET , SPL WND: GRAPHITE,3/4,300,7,FLEXITALL...",LB 7-1/2 ID X 8-1/2 OD CG 600 GASKETS FOR FLAN...,NaN,NaN,NaN,...,NaN,300,NaN,NaN,7,NaN,FLEXITALLIC,"1/16""""",NaN,INCONEL


Please check you local file system for output generated : File ----> final_output_df_file_import.csv
0.0 : 0.0 : 2.72977614402771


In [40]:
pred_output_df = pred_output_df.sort_values(['Noun','Modifier'], ascending=[True,True])
n_noun_modifier= pd.DataFrame()
n_noun_modifier = pred_output_df.loc[:,'Noun':'Modifier'].drop_duplicates()
n_noun_modifier.reset_index(drop=True, inplace=True)
writer = pd.ExcelWriter('final_output_df_file_import_excel.xlsx', engine='xlsxwriter')
#writer = pd.ExcelWriter('pandas_conditional.xlsx', engine='xlsxwriter')

for i in range(n_noun_modifier.shape[0]):
    noun=n_noun_modifier.loc[i,'Noun'].replace('/','_')
    modifier=n_noun_modifier.loc[i,'Modifier'].replace('/','_')
    df_name = str(noun) + "_" + str(modifier)
    name = n_noun_modifier.loc[i,'Noun'].replace(' ','_').replace('/','_') + "_" + n_noun_modifier.loc[i,'Modifier'].replace(' ','_').replace('/','_')
    df_taxonomy = pd.DataFrame(index=None)
    df_taxonomy = pred_output_df[pred_output_df['Noun'] == noun]
    df_taxonomy = df_taxonomy[df_taxonomy['Modifier'] == modifier]
    df_taxonomy.dropna(axis='columns',inplace=True,how='all')
    df_taxonomy = df_taxonomy.replace('NA','')
    df_taxonomy.reset_index(drop=True, inplace=True)
    df_taxonomy.columns = map(str.upper, df_taxonomy.columns)
    workbook  = writer.book
    df_taxonomy.to_excel(writer,sheet_name=name)
    worksheet = writer.sheets[name]
    start_row = 1
    start_col = 8
    end_row = df_taxonomy.shape[0]
    end_col = df_taxonomy.shape[1]
    format0 = workbook.add_format({'bg_color': '#FFA07A'})
    format1 = workbook.add_format({'bg_color': '#90EE90'})
    format2 = workbook.add_format({'font_color': '#FFFFFF','bg_color': '#0000FF'})
    cell_format = workbook.add_format()
    cell_format.set_align('left')
    cell_format.set_align('top')
    worksheet.conditional_format(start_row, start_col, end_row, end_col, {'type':'blanks', 'format': format0})
    worksheet.conditional_format(start_row, start_col, end_row, end_col, {'type':'no_blanks', 'format': format1})
    worksheet.conditional_format(start_row, start_col, end_row, end_col, {'format': cell_format})
    worksheet.conditional_format(0, 0, 0, end_col, {'type':'no_blanks', 'format': format2})
writer.save()
